# Heart Attack and Poverty Analysis

In [5]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import requests
from pprint import pprint
from census import Census
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
from requests.structures import CaseInsensitiveDict
import datetime
from scipy import stats


# File to Load (Remember to Change These)
path_poverty = Path("Resources/minnesota_poverty_data.csv")
path_heart = Path("Resources/heart_attack_hospitalizations_by_county.csv")

poverty_df = pd.read_csv(path_poverty)
heart_df = pd.read_csv(path_heart)
print(poverty_df['Year'])

0       2010
1       2010
2       2010
3       2010
4       2010
        ... 
1039    2021
1040    2021
1041    2021
1042    2021
1043    2021
Name: Year, Length: 1044, dtype: int64


In [9]:
# Need to break out the years in the heart data so we have individual years
heart_df['year']

27000                                                 Minnesota
27000                                                 Minnesota
27000                                                 Minnesota
27000                                                 Minnesota
27000                                                 Minnesota
                                                        ...    
St. Paul, MN 55164-0882                                     NaN
Phone: (651) 201-5900 or toll free: 1-800-205-4987          NaN
Fax: (651) 201-5926                                         NaN
E-mail: tracking.health@state.mn.us                         NaN
Web: http://health.mn.gov/mndata                            NaN
Name: year, Length: 716, dtype: object